In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split

In [25]:

dataset = Cityscapes("E:\CityScapes", split='train', mode='fine',
                     target_type='semantic',  transform=ToTensor(), target_transform = ToTensor())

train_dataset, val_dataset = random_split(dataset, [0.8,0.2])


In [27]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [28]:

from model import Model

model = Model().cuda()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [33]:
def train(dataloader, model, loss_fn, optimizer):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (X,_) in enumerate(dataloader):
        
        #predict
        pred = model(X)
        #Loss
        loss = loss_fn(pred, X)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [34]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (X,_) in enumerate(dataloader):
            
            pred = model(X)
            test_loss += loss_fn(pred, X).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")


In [35]:
epochs = 1 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor